In [97]:
import os
import pandas as pd
import arcpy
from arcpy import env
from arcpy.management import *
from arcpy.sa import *
from arcpy.da import *
from arcpy.conversion import *
from arcpy.analysis import *
import numpy as np

In [98]:
# tool function
# 创建渔网
def create_net(output_file,extent_file,cell_size):
    """参数说明：
        output_file: 输出文件
        extent_file: 输入范围文件
        cell_size: 生成的cell大小
    """
    proj = arcpy.Describe(extent_file).spatialReference
    extent_x_min = arcpy.Describe(extent_file).extent.XMin
    extent_y_min = arcpy.Describe(extent_file).extent.YMin
    extent_x_max = arcpy.Describe(extent_file).extent.XMax
    extent_y_max = arcpy.Describe(extent_file).extent.YMax
    with arcpy.EnvManager(outputCoordinateSystem = proj):
        arcpy.management.CreateFishnet(out_feature_class=output_file,origin_coord=f"{extent_x_min} {extent_y_min}",y_axis_coord=f"{extent_x_min} {extent_y_min+10}",cell_width=cell_size,cell_height=cell_size,number_rows=None,number_columns=None,corner_coord=f"{extent_x_max} {extent_y_max}",labels="NO_LABELS",template="#",geometry_type="POLYGON")
# 面转点
def polygon_point(in_feature, out_feature):
    """参数说明：
        in_feature: 输入面
        out_feature: 输出点
    """
    proj = arcpy.Describe(in_feature).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=proj):
        arcpy.management.FeatureToPoint(in_features=in_feature,out_feature_class=out_feature,point_location="INSIDE")

# 筛选点

def select_point(input_polygon,input_point,output_point):
    """参数说明：
        input_polygon: 选择面
        input_point: 待选择点
        output_point: 选择点
    """
    polygon_layer = "polygonLayer"
    point_layer = "pointLayer"
    arcpy.MakeFeatureLayer_management(input_polygon, polygon_layer)
    arcpy.MakeFeatureLayer_management(input_point, point_layer)
    arcpy.SelectLayerByLocation_management(in_layer=point_layer, overlap_type="INTERSECT", select_features=polygon_layer,search_distance=0,selection_type='NEW_SELECTION',invert_spatial_relationship="NOT_INVERT")
    # 导出选择的要素
    arcpy.CopyFeatures_management(point_layer, output_point)

# 添加字段
def add_field(input_table,field_name,field_type='TEXT'):
    """参数说明：
        input_table: 输入数据表
        field_name: 字段名
        field_type: 字段类型"""
    arcpy.AddField_management(input_table,field_name,field_type)
# 删除要素
def delete_feature(input_feature):
    arcpy.Delete_management(input_feature)
# 空间连接
def perform_spatial_join(target_layer_path, join_layer_path, output_layer_path, field_mapping_dict):
    """参数说明：
        target_layer_path: 目标图层路径
        join_layer_path: 连接图层路径
        output_layer_path: 输出图层路径
        field_mapping_dict: 字段映射字典
    """
    # 创建 FieldMappings 对象
    field_mappings = arcpy.FieldMappings()

    # 为每个字段对创建 FieldMap 并添加到 FieldMappings
    for target_field, join_field in field_mapping_dict.items():
        field_map = arcpy.FieldMap()
        
        # 添加目标字段和连接字段到FieldMap
        field_map.addInputField(target_layer_path, target_field)
        field_map.addInputField(join_layer_path, join_field)
        
        # 设置输出字段属性
        output_field = field_map.outputField
        output_field.name = target_field
        output_field.aliasName = target_field
        field_map.outputField = output_field

        # 将 FieldMap 添加到 FieldMappings
        field_mappings.addFieldMap(field_map)

    # 执行空间连接操作
    arcpy.analysis.SpatialJoin(target_layer_path, join_layer_path, output_layer_path,"JOIN_ONE_TO_ONE", "KEEP_ALL", field_mappings)


# 点采样
def sample_point(point_,raster_,out_name):
    """根据栅格采样点,输出为表格"""
    Sample(raster_,point_,out_name,"NEAREST", "OBJECTID", "CURRENT_SLICE", None, '', None, None, "ROW_WISE", "TABLE")
    return None

# 导出CSV
def export_csv(table_,out_path,out_name):
    """参数说明：
        table_: 输入数据表
        out_path: 输出路径
        out_name: 输出表名"""
    TableToTable(table_,out_path,out_name)
    return None

# 要素转点
def feature_to_point(input_layer,output_layer):
    """参数说明："""
    proj = arcpy.Describe(input_layer).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=proj):
        arcpy.management.FeatureToPoint(input_layer, output_layer, "INSIDE")

# 删除多余字段
def delete_extra_fields(input_feature_class, fields_list):
    """参数说明："""
    fields = arcpy.ListFields(input_feature_class)
    for field in fields:
        name = field.name
        type = field.type.upper()
        # 如果字段名不在删选列表中且类型不为OID和Geometry便删除这个字段
        if name not in fields_list and type not in ['OID','GEOMETRY']:
            arcpy.DeleteField_management(input_feature_class, [name])

In [99]:
# 设置工作空间
env.workspace = r'D:\ArcGISProjects\workspace\shbyq\features_data_vector.gdb\features_dy'
env.overwriteOutput=True

In [102]:
def create_point(input_layer):
    """生成虚拟点"""
    # 定义文件名称
    fish_net_name = "dunyun_grid"
    net_to_point_name = "dunyun_inside_point"
    select_point_name = "select_point"
    result_point_name = "join_point"
    # 渔网构建
    create_net(fish_net_name,input_layer,300)
    # 要素转点
    polygon_point(fish_net_name,net_to_point_name)
    # 按位置选择
    select_point(input_layer,net_to_point_name,select_point_name)
    # 添加字段
    field_list = ['TL','YL','TS','TZ']
    for one_field in field_list:
        # 添加字段
        add_field(input_table=select_point_name,field_name=one_field,field_type='TEXT')
    # 空间连接赋予地类属性
    # 定义目标要素图层和连接要素图层的路径
    fields_mapping = {
        "TL": "TL",
        "YL": "YL",
        "TS": "TS",
        "TZ": "TZ"
    }
    # 空间连接
    perform_spatial_join(select_point_name,input_layer,result_point_name,fields_mapping)
    # 删除多余数据
    for one_feature in [fish_net_name,net_to_point_name,select_point_name]:
        delete_feature(one_feature)

In [103]:
# 生成渔网点
create_point(input_layer="dy_tl")

In [104]:
# 生成面内部点
feature_to_point(input_layer="dy_tl",output_layer="dy_tl_dh_point")

In [106]:
# 内部点字段整理
# 添加字段
input_feature = "dy_tl_dh_point"
field_list = ['TL','YL','TS','TZ']
# calc_field = "修改代号"
# for one_field in field_list:
#     # 添加字段
#     add_field(input_table=input_feature,field_name=one_field,field_type='TEXT')
#     # 注意：这里使用了Python表达式
#     arcpy.CalculateField_management(input_feature, one_field, f"!{calc_field}!", "PYTHON3")
# 删除多余字段
delete_extra_fields(input_feature, field_list)


In [107]:
# 渔网点字段整理
# 添加字段
input_feature = "join_point"
field_list = ['TL','YL','TS','TZ']
# 删除多余字段
delete_extra_fields(input_feature, field_list)

In [108]:
# 为两个点位数据增加label
feature_1 = "join_point"
feature_2 = "dy_tl_dh_point"
for one_feature in [[feature_1,'fish_net'],[feature_2,'inner']]:
    features = one_feature[0]
    labels = one_feature[1]
    field_name = "label"
    # 添加字段
    add_field(input_table=features,field_name="label",field_type='TEXT')
    # 注意：这里使用了Python表达式
    arcpy.CalculateField_management(features, field_name, f"'{labels}'", "PYTHON3")


In [109]:
# 合并点位数据
feature_1 = "join_point"
feature_2 = "dy_tl_dh_point"
output_feature = "filter_result_point"
# 合并
arcpy.management.Merge([feature_1,feature_2],output_feature)
# 删除过渡文件
for one_feature in [feature_1,feature_2]:
    delete_feature(one_feature)

# 采样环节

In [14]:
# 对特征因子采样
# result table name
sample_csv_name = 'dy_frequency_filter_less_feature.csv'
# 选择用于采样的特征因子数据库
stander_raster_gdb = r"D:\ArcGISProjects\workspace\shbyq\feature_raster_file\features_data_dy.gdb"
env.workspace = stander_raster_gdb # 切换工作空间用于采样
# 用于采样的特征因子列表
feature_list = arcpy.ListRasters()
len(feature_list)

37

In [15]:
# 选择用于采样的要素类
point_data = r"D:\ArcGISProjects\workspace\shbyq\features_data_vector.gdb\features\filter_result_point"

# 用于存放采样表的数据库路径
sample_gdb_path = r"F:\cache_data\frequency_filter\dy"

In [31]:
# 采样点数据名称
sample_name =os.path.basename(point_data)
filed_list = [_.name for _ in arcpy.ListFields(point_data)]
print(filed_list)
# 需要保留的字段
# elements_yes = ['OBJECTID', 'TL','YL','TS','TZ']
elements_yes = ['OBJECTID', 'TZ']
filter_list = [_ for _ in filed_list if _ in elements_yes]
print(filter_list)

['OBJECTID', 'Shape', 'TL', 'YL', 'TS', 'TZ', 'label']
['OBJECTID', 'TZ']


In [17]:
# 使用Delete_management函数删除数据库中的所有内容
try:
    arcpy.Delete_management(os.path.join(sample_gdb_path,'tabledata.gdb'))
except:
    pass
# 再创建一个新的数据库
arcpy.management.CreateFileGDB(sample_gdb_path, "tabledata", "CURRENT")

<Result 'F:\\cache_data\\frequency_filter\\dy\\tabledata.gdb'>

In [18]:
# 逐个采样并保存到csv文件
gdb_path = os.path.join(sample_gdb_path,"tabledata.gdb")
for one_feature in feature_list:
    print(one_feature)
    sample_point(point_data,one_feature,os.path.join(gdb_path,one_feature))

DEM
AnalyticalHillshading
Aspect
ChannelNetworkBaseLevel
ChannelNetworkDistance
ClosedDepressions
ConvergenceIndex
LSFactor
PlanCurvature
ProfileCurvature
RelativeSlopePosition
Slope
TopographicWetnessIndex
TotalCatchmentArea
ValleyDepth
Contrast
Correlation
Dissimilarity
Entropy
Homogeneity
Mean
ndvi
PCA_0
PCA_1
SecondMoment
Variance
PRE
SRA
TMP
VAP
WIND
BIO
LON
LAT
PH
DZ
DL


In [19]:
# 导出CSV表用于分析
env.workspace = gdb_path # 切换工作空间用于导出csv文件

In [34]:
# 读取数据表并保存到csv文件
result_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(point_data,filter_list+['label']))
result_df.rename(columns={"OBJECTID":sample_name},inplace=True)

In [35]:
#  读取每个表的最后一个字段的数据,存储每个表的最后一个字段的数据
for table in feature_list:
    print(table)
    # 将表转换为pandas数据帧
    # df = pd.DataFrame(arcpy.da.TableToNumPyArray(table, "*"))  # 确保数据表中无空值
    # df = pd.DataFrame(arcpy.da.TableToNumPyArray(table, "*",skip_nulls=False,null_value=99))  # 确保数据表中无空值
    df = pd.DataFrame(arcpy.da.TableToNumPyArray(table, "*",null_value=0))  # 确保数据表中无空值
    # 提取最后一个字段的数据
    merged_df = df[[sample_name, df.columns[-1]]]
    # 合并
    result_df = pd.merge(result_df, merged_df, on=[sample_name])
# 保存到csv文件
result_df.rename(columns=dict(zip(result_df.columns[-len(feature_list):], feature_list)),inplace=True)
result_df.drop(result_df.columns[0],axis=1,inplace=True)
# result_df.rename(columns={"PH":"PH_T"},inplace=True)
result_df.to_csv(os.path.join(sample_gdb_path,sample_csv_name),index=False,encoding='utf-8')

DEM
AnalyticalHillshading
Aspect
ChannelNetworkBaseLevel
ChannelNetworkDistance
ClosedDepressions
ConvergenceIndex
LSFactor
PlanCurvature
ProfileCurvature
RelativeSlopePosition
Slope
TopographicWetnessIndex
TotalCatchmentArea
ValleyDepth
Contrast
Correlation
Dissimilarity
Entropy
Homogeneity
Mean
ndvi
PCA_0
PCA_1
SecondMoment
Variance
PRE
SRA
TMP
VAP
WIND
BIO
LON
LAT
PH
DZ
DL


In [36]:
df = pd.read_csv(os.path.join(sample_gdb_path,sample_csv_name))
df

,TZ,label,DEM,AnalyticalHillshading,Aspect,ChannelNetworkBaseLevel,ChannelNetworkDistance,ClosedDepressions,ConvergenceIndex,LSFactor,...,SRA,TMP,VAP,WIND,BIO,LON,LAT,PH,DZ,DL
0,13d,fish_net,961.5,0.706572,0.231111,827.27014,134.229860,0.0,16.303608,1.840937,...,13154.333,15.766666,1.555833,1.658333,194.86105,443245.0,2861300.0,5.355657,9,7
1,106,fish_net,888.4,0.426612,6.042653,870.42035,17.979675,0.0,2.823421,13.958138,...,13154.333,15.766666,1.555833,1.658333,194.86105,443545.0,2861300.0,5.901145,9,7
2,13d,fish_net,870.8,1.131889,1.264116,835.30994,35.490050,0.0,-2.361449,18.948938,...,13154.333,15.766666,1.555833,1.658333,194.86105,443845.0,2861300.0,5.594585,9,3
3,13d,fish_net,1021.6,0.186124,5.645218,940.00836,81.591610,0.0,4.197873,13.658210,...,13140.667,15.600000,1.540833,1.683333,195.54823,444445.0,2861300.0,5.476191,9,7
4,14d,fish_net,853.4,0.811739,4.231506,814.98510,38.414917,0.0,1.524923,20.288977,...,13150.667,15.750000,1.551667,1.675000,194.79027,442045.0,2861600.0,5.976630,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26911,26,inner,984.5,0.905172,3.419885,938.43884,46.061157,0.0,-2.104909,3.448655,...,12870.833,15.183333,1.489167,1.708333,189.20451,451555.0,2920805.0,5.539885,10,9
26912,68,inner,1308.2,0.549022,4.897726,1283.38370,24.816284,0.0,8.089989,5.066950,...,12783.750,13.858334,1.375000,1.900000,191.19537,428765.0,2906345.0,5.938250,4,7
26913,68,inner,1384.3,1.375126,2.949196,1325.74930,58.550780,0.0,3.121301,15.663603,...,12785.083,13.866667,1.380000,1.908333,191.05931,429425.0,2906145.0,5.412776,4,9
26914,61f,inner,1358.7,0.766001,4.187584,1345.38040,13.319580,0.0,2.402896,11.407861,...,12787.667,13.591666,1.354167,1.925000,191.71070,428495.0,2905255.0,5.489189,4,7


In [3]:
file_path = r"D:\ArcGISProjects\workspace\shbyq\features_data_vector.gdb\features\dy_tl"

In [4]:
field_list = arcpy.ListFields(file_path)

In [5]:
for one_field in field_list:
    print(one_field.name)

OBJECTID
Shape
土种代
Shape_Leng
土类
亚类
土属
土种
土种编号
Shape_Length
Shape_Area


In [6]:
df2 = pd.DataFrame(arcpy.da.TableToNumPyArray(r"D:\ArcGISProjects\workspace\shbyq\features_data_vector.gdb\features\dy_tl", ["土类","亚类","土属","土种","土种编号"]))  # 确保数据表中无空值

In [7]:
# 对每个唯一的土种编号生成一个字典
def aggregate_rows(group):
    records = group[['土类', '亚类', '土属', '土种']].to_dict('records')
    # 假设每个土种编号下的记录是相同的，这里我们取第一个记录
    return records[0] if records else {}

# 使用groupby按照`土种编号`聚合，并应用自定义函数
soil_dict = df2.groupby('土种编号').apply(aggregate_rows).to_dict()

In [8]:
soil_dict,len(soil_dict)

({'101': {'土类': '水稻土', '亚类': '淹育型水稻土', '土属': '淹育型潮沙田', '土种': '潮板沙田'},
  '102': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型黄泥田APWC或APBC', '土种': '黄泥田'},
  '103': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型黄泥田APWC或APBC', '土种': '黄胶泥田'},
  '104': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型黄泥田APWC或APBC', '土种': '小黄泥田'},
  '105': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型黄泥田APWC或APBC', '土种': '黄油沙泥田'},
  '106': {'土类': '水稻土',
   '亚类': '潴育型水稻土',
   '土属': '潴育型石灰性大眼泥田ARWC或APBC',
   '土种': '大眼泥田'},
  '107': {'土类': '水稻土',
   '亚类': '潴育型水稻土',
   '土属': '潴育型石灰性大眼泥田ARWC或APBC',
   '土种': '龙凤大眼泥田'},
  '108': {'土类': '水稻土',
   '亚类': '潴育型水稻土',
   '土属': '潴育型石灰性大眼泥田ARWC或APBC',
   '土种': '大眼黄泥田'},
  '112': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型潮泥田APWC或APWG', '土种': '潮沙田'},
  '113': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型潮泥田APWC或APWG', '土种': '潮沙泥田'},
  '114': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型潮泥田APWC或APWG', '土种': '潮泥田'},
  '115': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型潮泥田APWC或APWG', '土种': '潮油沙泥田'},
  '116': {'土类': '水稻土

In [9]:
excel_path = r"C:\Users\Runker\Desktop\土壤属性表.xls"

In [51]:
df_excel = pd.read_excel(excel_path)
# 将土种编号一列设置为字符串类型
df_excel['土种编号'] = df_excel['土种编号'].astype(str)
df_excel

,土类,亚类,土属,土种,土种编号
0,山地灌丛草甸土,山地灌丛草甸土,山地灌丛草甸土,薄腐殖层中土层山地灌丛草甸土,1e
1,山地灌丛草甸土,山地灌丛草甸土,山地灌丛草甸土,薄腐殖薄土层山地灌丛草甸土,1f
2,黄壤,黄壤,铁质黄壤,薄腐殖层薄土层铁质黄壤,11f
3,黄壤,黄壤,硅质黄壤,厚腐殖层中土层硅质黄壤,12b
4,黄壤,黄壤,硅质黄壤,厚腐殖层薄土层硅质黄壤,12e
...,...,...,...,...,...
85,水稻土,侧渗型水稻土,白泥田A—P E—E(C)或A—P—E(C),白沙泥田,137
86,水稻土,矿毒田,锈毒田MA—MG MA—P—C,锈水田,139
87,水稻土,矿毒田,锈毒田MA—MG MA—P—C,煤锈田,140
88,水稻土,矿毒田,锈毒田MA—MG MA—P—C,烂锈田,141


In [52]:
# 使用groupby按照`土种编号`聚合，并应用自定义函数
soil_excel = df_excel.groupby('土种编号').apply(aggregate_rows).to_dict()

In [53]:
soil_excel,len(soil_excel)

({'101': {'土类': '水稻土', '亚类': '淹育型水稻土', '土属': '淹育型潮沙田', '土种': '潮板沙田'},
  '102': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型黄泥田APWC或APBC', '土种': '黄泥田'},
  '103': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型黄泥田APWC或APBC', '土种': '黄胶泥田'},
  '104': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型黄泥田APWC或APBC', '土种': '小黄泥田'},
  '105': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型黄泥田APWC或APBC', '土种': '黄油沙泥田'},
  '106': {'土类': '水稻土',
   '亚类': '潴育型水稻土',
   '土属': '潴育型石灰性大眼泥田ARWC或APBC',
   '土种': '大眼泥田'},
  '107': {'土类': '水稻土',
   '亚类': '潴育型水稻土',
   '土属': '潴育型石灰性大眼泥田ARWC或APBC',
   '土种': '龙凤大眼泥田'},
  '108': {'土类': '水稻土',
   '亚类': '潴育型水稻土',
   '土属': '潴育型石灰性大眼泥田ARWC或APBC',
   '土种': '大眼黄泥田'},
  '109': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型紫泥田', '土种': '浅紫泥田'},
  '112': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型潮泥田APWC或APWG', '土种': '潮沙田'},
  '113': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型潮泥田APWC或APWG', '土种': '潮沙泥田'},
  '114': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型潮泥田APWC或APWG', '土种': '潮泥田'},
  '115': {'土类': '水稻土', '亚类': '

In [54]:
# 合并两个字典
result_dict = {**soil_dict, **soil_excel}
result_dict,len(result_dict)    

({'101': {'土类': '水稻土', '亚类': '淹育型水稻土', '土属': '淹育型潮沙田', '土种': '潮板沙田'},
  '102': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型黄泥田APWC或APBC', '土种': '黄泥田'},
  '103': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型黄泥田APWC或APBC', '土种': '黄胶泥田'},
  '104': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型黄泥田APWC或APBC', '土种': '小黄泥田'},
  '105': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型黄泥田APWC或APBC', '土种': '黄油沙泥田'},
  '106': {'土类': '水稻土',
   '亚类': '潴育型水稻土',
   '土属': '潴育型石灰性大眼泥田ARWC或APBC',
   '土种': '大眼泥田'},
  '107': {'土类': '水稻土',
   '亚类': '潴育型水稻土',
   '土属': '潴育型石灰性大眼泥田ARWC或APBC',
   '土种': '龙凤大眼泥田'},
  '108': {'土类': '水稻土',
   '亚类': '潴育型水稻土',
   '土属': '潴育型石灰性大眼泥田ARWC或APBC',
   '土种': '大眼黄泥田'},
  '112': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型潮泥田APWC或APWG', '土种': '潮沙田'},
  '113': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型潮泥田APWC或APWG', '土种': '潮沙泥田'},
  '114': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型潮泥田APWC或APWG', '土种': '潮泥田'},
  '115': {'土类': '水稻土', '亚类': '潴育型水稻土', '土属': '潴育型潮泥田APWC或APWG', '土种': '潮油沙泥田'},
  '116': {'土类': '水稻土

In [55]:
file_path2 = r'D:\ArcGISProjects\workspace\shbyq\features_data_vector.gdb\features\dy_tl_dh'

In [56]:
for one_name in arcpy.ListFields(file_path2):
    print(one_name.name)

OBJECTID
Shape
土种代
修改代号
修改过
Shape_Length
Shape_Area
TL
YL
TS
TZ


In [67]:
code_block =f"""
def get_soil_class(soil_code):
    
    soil_dict = {result_dict}
    if soil_code in soil_dict:
        return soil_dict[soil_code]['土类']
    else:
        return "NONE"
"""

In [58]:
expression = "get_soil_class(!修改代号!)"

In [68]:
field_name = "TL"

In [44]:
# 使用CalculateField_management执行更新
add_field(file_path2, field_name)

In [69]:
arcpy.CalculateField_management(in_table=file_path2, field=field_name, expression=expression, expression_type="PYTHON3", code_block=code_block)

<Result 'D:\\ArcGISProjects\\workspace\\shbyq\\features_data_vector.gdb\\features\\dy_tl_dh'>

In [5]:
df3 = pd.DataFrame(arcpy.da.TableToNumPyArray(file_path2, ["TL","YL","TS","TZ","修改代号"]))

In [6]:
df3['TL'].value_counts

<bound method IndexOpsMixin.value_counts of 0       水稻土
1       水稻土
2        黄壤
3       水稻土
4        黄壤
       ... 
1613     黄壤
1614    石灰土
1615    石灰土
1616    石灰土
1617    水稻土
Name: TL, Length: 1618, dtype: object>

In [8]:
df_none = df3[df3['TZ'] == 'NONE']
df_none

,TL,YL,TS,TZ,修改代号
140,NONE,NONE,NONE,NONE,137
149,NONE,NONE,NONE,NONE,65
249,NONE,NONE,NONE,NONE,60
254,NONE,NONE,NONE,NONE,137
304,NONE,NONE,NONE,NONE,12c
359,NONE,NONE,NONE,NONE,63
451,NONE,NONE,NONE,NONE,65
481,NONE,NONE,NONE,NONE,138
510,NONE,NONE,NONE,NONE,65
545,NONE,NONE,NONE,NONE,18d


In [11]:
# 获取唯一值
a = df_none['修改代号'].unique().tolist()
print(a),print(len(a))

['137', '65', '60', '12c', '63', '138', '18d', '61d', '72.5', '15', '2b', '11e', '65f', '132', '64f', '20', '74d', '2f', '4d', '4f', '67', '24', '4e']
23


(None, None)

In [84]:
search_dict = {0: ['101'],
1: ['102'],
 2: ['103'],
 3: ['104'],
 4: ['105'],
 5: ['106'],
 6: ['107'],
 7: ['108'],
 8: ['112'],
 9: ['113'],
 10: ['114'],
 11: ['115'],
 12: ['116'],
 13: ['117'],
 14: ['118'],
 15: ['119'],
 16: ['11e'],
 17: ['120'],
 18: ['125'],
 19: ['126'],
 20: ['129'],
 21: ['12b'],
 22: ['12c'],
 23: ['12d'],
 24: ['12e'],
 25: ['12f'],
 26: ['130'],
 27: ['131'],
 28: ['132'],
 29: ['135'],
 30: ['136'],
 31: ['137'],
 32: ['138'],
 33: ['139'],
 34: ['13d'],
 35: ['13e'],
 36: ['13f'],
 37: ['141'],
 38: ['143'],
 39: ['14d'],
 40: ['14e'],
 41: ['14f'],
 42: ['15'],
 43: ['15d'],
 44: ['15e'],
 45: ['15f'],
 46: ['18d'],
 47: ['1e'],
 48: ['1f'],
 49: ['20'],
 50: ['21'],
 51: ['22'],
 52: ['23'],
 53: ['24'],
 54: ['26'],
 55: ['29'],
 56: ['2b'],
 57: ['2f'],
 58: ['30'],
 59: ['31'],
 60: ['32'],
 61: ['33'],
 62: ['34'],
 63: ['4d'],
 64: ['4e'],
 65: ['4f'],
 66: ['60'],
 67: ['61d'],
 68: ['61e'],
 69: ['61f'],
 70: ['63'],
 71: ['63b'],
 72: ['63d'],
 73: ['63e'],
 74: ['63f'],
 75: ['64d'],
 76: ['64f'],
 77: ['65'],
 78: ['65d'],
 79: ['65e'],
 80: ['65f'],
 81: ['67'],
 82: ['68'],
 83: ['69'],
 84: ['70'],
 85: ['72'],
 86: ['72.5'],
 87: ['74d'],
 88: ['76e'],
 89: ['77.5'],
 90: ['79'],
 91: ['82'],
 92: ['86'],
 93: ['90'],
 94: ['91'],
 95: ['92'],
 96: ['93'],
 97: ['94'],
 98: ['95'],
 99: ['96'],
 100: ['97'],
 101: ['98']}

In [70]:
result_path = r'D:\ArcGISProjects\workspace\shbyq\MyProject.gdb\RasterT_predict1'

In [71]:
for one_field in arcpy.ListFields(result_path):
    print(one_field.name)

OBJECTID
Shape
Id
gridcode
Shape_Length
Shape_Area


In [72]:
# # 添加字段
# add_field(result_path, 'TL', 'TEXT')
# add_field(result_path, 'YL', 'TEXT')
# add_field(result_path, 'TS', 'TEXT')
# add_field(result_path, 'TZ', 'TEXT')

In [94]:
code_block =f"""
def get_soil_class(soil_code):
    search_dict = {search_dict}
    soil_dict = {result_dict}
    check_code = search_dict[soil_code][0]
    if check_code in soil_dict:
        return soil_dict[check_code]['土类']
    else:
        return "NONE"
"""
field_name = 'TL'

In [74]:
expression = "get_soil_class(!gridcode!)"

In [95]:
arcpy.CalculateField_management(in_table=result_path, field=field_name, expression=expression, expression_type="PYTHON3", code_block=code_block)

<Result 'D:\\ArcGISProjects\\workspace\\shbyq\\MyProject.gdb\\RasterT_predict1'>